# Proyecto Final Área de Imágenes
### EL5206 Laboratorio de Inteligencia Computacional y Robótica

Integrantes:
* José Díaz
* Luis Jiménez

El proyecto a realizar consiste en el desarrollo e implementación de un
algoritmo de búsqueda de imágenes del tipo CBIR, el cual hará comparaciones entre
una imagen de consulta e imágenes en una base de datos a través de sus vectores de características.
Para esta implementación se utilizarán dos extractores: uno clásico (HOG) y otro
basado en redes convolucionales (CNN).

### Librerías útiles

Carga de librerías útiles para el desarrollo del proyecto.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch
from PIL import Image
from torchvision import transforms
import os
from tqdm.notebook import tqdm
from skimage.feature import hog
from skimage import data, exposure

### 1. Extracción de características

Se generarán dos funciones auxiliares. Una función que extrae
el vector de características con el extractor HOG, y otra función
que utiliza el extractor CNN.

### EXTRACCIÓN TEMPORAL, SE CAMBIARÁ

In [3]:
#hog
r_path = 'jpg/'
w_path = 'jpg_hog/'

for file in tqdm(os.listdir(r_path)):
    image_path = r_path + file
    image = Image.open(image_path)
    image = image.resize((224,224))

    hog_vect = hog(image, orientations=16, pixels_per_cell=(8, 8),
                    cells_per_block=(1, 1), feature_vector=True, multichannel=True)

    vect_name = w_path + 'hog_' + file[:-4]
    #print(type(hog_vect))
    #print(hog_vect.shape)
    np.save(vect_name, hog_vect)

In [4]:
#cnn
model = torch.hub.load('pytorch/vision:v0.6.0', 'resnet18', pretrained=True)
model = torch.nn.Sequential(*(list(model.children())[:-1]))

Downloading: "https://github.com/pytorch/vision/archive/v0.6.0.zip" to C:\Users\Luis Jiménez/.cache\torch\hub\v0.6.0.zip


ImportError: cannot import name 'resnext50_32x4d' from 'torchvision.models.resnet' (c:\users\luis jiménez\desktop\importante\u\lab_inteligencia\proyecto\proyecto_final_el5206\venv\lib\site-packages\torchvision\models\resnet.py)

In [ ]:
preprocess = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
 # create a mini-batch as expected by the model

r_path = 'jpg/'
w_path = 'jpg_cnn/'

# move the input and model to GPU for speed if available
if torch.cuda.is_available():
    input_batch = input_batch.to('cuda')
    model.to('cuda')

with torch.no_grad():
    for file in tqdm(os.listdir(r_path)):
        image_path = r_path + file
        image = Image.open(image_path)
        image = image.resize((224,224))
        input_tensor = preprocess(image)
        input_batch = input_tensor.unsqueeze(0)

        output = model(input_batch)
        output = output.numpy().ravel()
        vect_name = w_path + 'cnn_' + file[:-4]
        #print(type(hog_vect))
        #print(hog_vect.shape)
        np.save(vect_name, output)

#### Extracción HOG

In [ ]:
#FUNCIÓN DE EXTRACCION HOG. ADAPTAR PARA QUE EXTRAIGA A UNA
#SOLA IMAGEN
#def feature_hog(img_path)




#### Extracción CNN

In [ ]:
#AQUÍ IMPLEMENTAR FUNCIÓN PARA EXTRACCIÓN CNN DE UNA SOLA IMAGEN
#def feature_cnn(img_path)

Utilzando las dos funciones auxiliares anteriores,
la función a continuación calcula el vector de características asociado
a una imagen. Los parámetros de esta función son la imagen en cuestión y el tipo de extractor
de características, el cual se puede elegir entre HOG y CNN.

In [ ]:
#FUNCIÓN QUE COMBINA LAS DOS ANTERIORES Y EXTRAE CARACTERISTICAS
'''def feature_vector(img_path, extractor):

    if extractor == 'HOG':

        feat_vector = feature_hog(img_path)

    elif extractor == 'CNN':

        feat_vector = feature_cnn(img_path)

    return feat_vector
'''

A continuación, se extraen las características de cada imagen de la
base de datos disponible.

In [ ]:
#ACÁ USAR LOS CICLOS FOR Y LA FUNCIÓN ANTERIOR, Y GUARDAR EN UNA
#VARIABLE LOS VECTORES EXTRAÍDOS

### 2. Métricas

Se definen métricas de comparación entre vectores de
características de imágenes, así como medidas para generar
un ranking de imágenes similares a una imagen de consulta
introducida por el usuario.

#### Distancia euclideana

In [3]:
def dist_euc(X, Y):

    dist = np.linalg.norm(X-Y)
    return dist

#### Distancia chi-cuadrado [1]

In [4]:
def dist_chi2(X, Y):

    chi = 0.5 * np.sum([((x - y) ** 2) / (x + y)
                      for (x, y) in zip(X, Y)])
    return chi

##### Pruebas de dist_euc y dist_chi2

In [5]:
X = np.array([1,2,3])
Y = np.array([5,6,7])
print(dist_euc(X, Y), dist_chi2(X, Y))

6.928203230275509 3.133333333333333


#### Función para elegir tipo de distancia

In [ ]:
def dist_vectors(X, Y, dist_type):

    if dist_type == 'euc':

        dist = dist_euc(X, Y)

    elif dist_type == 'chi2':

        dist = dist_chi2(X, Y)

    return dist

### 3. Comparación de una imagen de consulta con la BBDD

En esta sección, para cada imagen de consulta, se calcula la distancia entre
su vector de características con todos los vectores de características de
la base de datos.

RECORDATORIO: ID_imagen = 00 -> CONSULTA

DISTINTO DE 00 ES BBDD

### 4. Ordenamiento de imágenes por relevancia (medida de Ranking)

A continuación, para cada imagen de consulta, se ordenan de menor a mayor
las distancias obtenidas al hacer la comparación con la base de datos.

para cada imagen "query",
calculaba sus distancias con todas las imagenes en la
base de datos, las ordenaba, y calculaba el ranking
sacando el promedio de la posición en que quedaron
las imágenes de la misma clase a la query

In [ ]:
#ORDENAMIENTO DE DISTANCIAS

Luego, con las imágenes ordenadas, se calcula una medida de ranking,
que entrega la posición promedio de las imágenes que entrega el buscador
al realizar la consulta de una imagen.

Las imágenes consideradas relevantes para la consulta de una imagen particular
corresponden a imágenes de la base de datos que pertenecen a la misma
clase que la imagen de consulta.

In [ ]:
#def rank(image, Nrel):



### 5. Comparación de métodos de extracción

Se necesita evaluar la robustez de los métodos de extracción utilizados
en este proyecto, y una forma de compararlos es mediante la normalización
de la medida de Ranking que se utilizó anteriormente.

In [ ]:
#def rank_norm(image, N, Nrel):


### 6. Ejemplos de uso del algoritmo

A continuación se muestran algunos ejemplos de consulta de imágenes
y la respuesta del método, cambiando tipos de extractores y métricas
de distancias utilizadas.

### 7. Optimización algorítmica del método

### Referencias utilizadas

[1] https://www.geeksforgeeks.org/chi-square-distance-in-python/

